In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import pymongo
from pymongo import MongoClient

In [2]:
# Connection to mongo
client = MongoClient('mongodb+srv://<user>:<password>@cluster0.l3pqt.mongodb.net/MSA?retryWrites=true&w=majority')
# Select database
db = client['MSA']
# see list of collections
client.MSA.list_collection_names()

['CPI_RegionUrban_raw',
 'LaborForce_clean',
 'employment_prediction',
 'population_clean',
 'GDP_monthly',
 'population_predicted_2024',
 'employment_clean',
 'AvgWeeklyEarningsAllEmp_raw',
 'GDP_raw',
 'unemployment_ROC_2024',
 'employment_raw',
 'OES_raw',
 'msa_codes',
 'unemployment_predicted_2024',
 'arima_pop_pred_score',
 'arima_unem_pred_score',
 'population_raw',
 'arima_emp_pred_score',
 'Population_monthly',
 'Employment_clean',
 'LaborForce_raw',
 'Employment_raw',
 'unemployment_raw',
 'GDP_clean',
 'UnemploymentRate_clean',
 'UnemploymentRate_raw',
 'Predicted_2024_ROC_rank_total',
 'Predicted_2024_ROC_rank_total2',
 'GDP_predicted_2024']

In [3]:
# Select the collection with needed data
gdp = db.GDP_clean
df = pd.DataFrame(list(gdp.find()))
print(df.dtypes)
df

_id     object
CBSA     int64
2010     int64
2011     int64
2012     int64
2013     int64
2014     int64
2015     int64
2016     int64
2017     int64
2018     int64
2019     int64
dtype: object


,_id,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,60329bde9efa42f38169dc3c,14010,10125712,10344962,10781922,10894563,12406870,13579971,14215189,14266863,15313534,15348042
1,60329bde9efa42f38169dc55,16700,27410195,29228911,31499192,32022303,33904385,36299014,38722725,40807777,43313850,45585291
2,60329bde9efa42f38169dc58,16860,21693651,22869065,24182289,25000279,25457470,26880790,27745927,28892743,30071077,31239821
3,60329bde9efa42f38169dc68,18580,18450337,20849475,21248470,21527986,20594117,20609947,19853738,21428439,23576086,23673074
4,60329bde9efa42f38169dc6e,19180,2989513,3110378,3278839,3316481,3402008,3344909,3339878,3360536,3453415,3581970
...,...,...,...,...,...,...,...,...,...,...,...,...
379,60329bde9efa42f38169dd51,42140,5842150,5824572,5903877,6036941,6114395,6302738,6381240,6453142,6715055,7013692
380,60329bde9efa42f38169dd5d,43580,7434099,7296955,7198836,7266016,7368954,7906228,8107411,8058981,8506767,8796560
381,60329bde9efa42f38169dd62,44100,9891926,10124667,10161515,10364612,10742709,10960442,11124600,11449013,12162309,12526094
382,60329bde9efa42f38169dd6a,45060,32825188,32782974,34299459,35577814,36833391,38336972,38446915,39617908,41301663,43390586


In [4]:
df.drop(columns='_id', inplace=True)
df

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,14010,10125712,10344962,10781922,10894563,12406870,13579971,14215189,14266863,15313534,15348042
1,16700,27410195,29228911,31499192,32022303,33904385,36299014,38722725,40807777,43313850,45585291
2,16860,21693651,22869065,24182289,25000279,25457470,26880790,27745927,28892743,30071077,31239821
3,18580,18450337,20849475,21248470,21527986,20594117,20609947,19853738,21428439,23576086,23673074
4,19180,2989513,3110378,3278839,3316481,3402008,3344909,3339878,3360536,3453415,3581970
...,...,...,...,...,...,...,...,...,...,...,...
379,42140,5842150,5824572,5903877,6036941,6114395,6302738,6381240,6453142,6715055,7013692
380,43580,7434099,7296955,7198836,7266016,7368954,7906228,8107411,8058981,8506767,8796560
381,44100,9891926,10124667,10161515,10364612,10742709,10960442,11124600,11449013,12162309,12526094
382,45060,32825188,32782974,34299459,35577814,36833391,38336972,38446915,39617908,41301663,43390586


In [5]:
# for loop to predict 2024 values
predictions = []
for i in range(0,384):
    y = df.iloc[i, 1:].values
    series = pd.Series(y, dtype='int')
    model = ARIMA(series, order=(1, 1, 1))
    model_fit = model.fit()
    pred = model_fit.forecast(5)
    predictions.append(pred.values.tolist()[-1])
df['2024'] = predictions

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization faile

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

In [8]:
df

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2024
0,14010,10125712,10344962,10781922,10894563,12406870,13579971,14215189,14266863,15313534,15348042,1.815899e+07
1,16700,27410195,29228911,31499192,32022303,33904385,36299014,38722725,40807777,43313850,45585291,5.564449e+07
2,16860,21693651,22869065,24182289,25000279,25457470,26880790,27745927,28892743,30071077,31239821,3.651152e+07
3,18580,18450337,20849475,21248470,21527986,20594117,20609947,19853738,21428439,23576086,23673074,2.369156e+07
4,19180,2989513,3110378,3278839,3316481,3402008,3344909,3339878,3360536,3453415,3581970,3.641137e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
379,42140,5842150,5824572,5903877,6036941,6114395,6302738,6381240,6453142,6715055,7013692,7.659942e+06
380,43580,7434099,7296955,7198836,7266016,7368954,7906228,8107411,8058981,8506767,8796560,9.522426e+06
381,44100,9891926,10124667,10161515,10364612,10742709,10960442,11124600,11449013,12162309,12526094,1.397789e+07
382,45060,32825188,32782974,34299459,35577814,36833391,38336972,38446915,39617908,41301663,43390586,4.920274e+07


In [11]:
df['2024'] = df['2024'].astype('int')
df.head()

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2024
0,14010,10125712,10344962,10781922,10894563,12406870,13579971,14215189,14266863,15313534,15348042,18158992
1,16700,27410195,29228911,31499192,32022303,33904385,36299014,38722725,40807777,43313850,45585291,55644492
2,16860,21693651,22869065,24182289,25000279,25457470,26880790,27745927,28892743,30071077,31239821,36511515
3,18580,18450337,20849475,21248470,21527986,20594117,20609947,19853738,21428439,23576086,23673074,23691561
4,19180,2989513,3110378,3278839,3316481,3402008,3344909,3339878,3360536,3453415,3581970,3641136


In [12]:
df['2024_GDP_ROC'] = (df['2024']-df['2019'])/df['2019'] *100
df.head()

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2024,2024_GDP_ROC
0,14010,10125712,10344962,10781922,10894563,12406870,13579971,14215189,14266863,15313534,15348042,18158992,18.314714
1,16700,27410195,29228911,31499192,32022303,33904385,36299014,38722725,40807777,43313850,45585291,55644492,22.066769
2,16860,21693651,22869065,24182289,25000279,25457470,26880790,27745927,28892743,30071077,31239821,36511515,16.874917
3,18580,18450337,20849475,21248470,21527986,20594117,20609947,19853738,21428439,23576086,23673074,23691561,0.078093
4,19180,2989513,3110378,3278839,3316481,3402008,3344909,3339878,3360536,3453415,3581970,3641136,1.651773


In [21]:
df_pred = df[['CBSA', '2024_GDP_ROC']].copy()
df_pred

,CBSA,2024_GDP_ROC
0,14010,18.314714
1,16700,22.066769
2,16860,16.874917
3,18580,0.078093
4,19180,1.651773
...,...,...
379,42140,9.214120
380,43580,8.251703
381,44100,11.590181
382,45060,13.394975


In [22]:
df_pred['GDP_Score'] = df_pred['2024_GDP_ROC'].rank(ascending=True)

In [26]:
df_pred.head()

,CBSA,2024_GDP_ROC,GDP_Score
0,14010,18.314714,316.0
1,16700,22.066769,370.0
2,16860,16.874917,287.0
3,18580,0.078093,44.0
4,19180,1.651773,72.0


In [27]:
# create new collection in mongo
arima_GDP_pred_score = db.arima_GDP_pred_score

In [28]:
df_dict = df_pred.to_dict(orient='records')

In [29]:
arima_GDP_pred_score.insert_many(df_dict)